In [1]:
# Cellule 1 — Imports, config, chemins

# ============================================================
# Notebook 10 — Inference V13 sur test_eliva25 (images prétraitées)
# Génère submission_v13.csv pour Kaggle
# ============================================================

import os
from pathlib import Path

import numpy as np
import pandas as pd
import tensorflow as tf

from tqdm import tqdm
from tensorflow.keras.applications.efficientnet_v2 import preprocess_input

# --- Config GPU / mixed precision (comme notebook 05) ---
from tensorflow.keras import mixed_precision

SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

policy = mixed_precision.Policy("mixed_float16")
mixed_precision.set_global_policy(policy)
print("Politique de précision mixte :", mixed_precision.global_policy())

gpus = tf.config.list_physical_devices("GPU")
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_memory_growth(gpu, True)
    print(f"GPU détecté ({len(gpus)}) — croissance mémoire activée.")
else:
    print("Aucun GPU détecté → CPU")

# --- Constantes cohérentes avec V13 ---
IMG_SIZE = 300
BATCH_SIZE = 8
AUTOTUNE = tf.data.AUTOTUNE

# --- Chemins (adapte BASE_DIR si besoin) ---
BASE_DIR = Path("/home/fabrice/Boneage")

PREPROC_TEST_DIR = BASE_DIR / "preprocessed_test_eliva25"
TEST_CSV_PATH    = BASE_DIR / "test_eliva25.csv"

MODEL_V13_PATH   = BASE_DIR / "model_v13_preproc_v3.keras"
SUBMISSION_PATH  = BASE_DIR / "submission_v13.csv"

print("\n=== Chemins inference V13 ===")
print("BASE_DIR          :", BASE_DIR)
print("Images prétraitées:", PREPROC_TEST_DIR)
print("CSV test          :", TEST_CSV_PATH)
print("Modèle V13        :", MODEL_V13_PATH)
print("Fichier submission:", SUBMISSION_PATH)


2025-12-01 20:00:25.961038: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-12-01 20:00:25.986955: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-12-01 20:00:26.004636: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-12-01 20:00:26.010140: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-12-01 20:00:26.034143: I tensorflow/core/platform/cpu_feature_guar

Politique de précision mixte : <DTypePolicy "mixed_float16">
GPU détecté (1) — croissance mémoire activée.

=== Chemins inference V13 ===
BASE_DIR          : /home/fabrice/Boneage
Images prétraitées: /home/fabrice/Boneage/preprocessed_test_eliva25
CSV test          : /home/fabrice/Boneage/test_eliva25.csv
Modèle V13        : /home/fabrice/Boneage/model_v13_preproc_v3.keras
Fichier submission: /home/fabrice/Boneage/submission_v13.csv


I0000 00:00:1764619227.286603   15183 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764619227.549423   15183 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764619227.549502   15183 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.


In [2]:
# ============================================================
# Cellule 2 — Chargement test_eliva25.csv et préparation des chemins
# ============================================================

# 1) Charger le CSV test
test_df = pd.read_csv(TEST_CSV_PATH)
print("Aperçu test_eliva25.csv :")
display(test_df.head())

print("\nNombre de lignes dans test_eliva25.csv :", len(test_df))

# 2) Conversion de la colonne 'male' en float32 (0 / 1)
def convert_male_to_float(x):
    # gère True/False, 'TRUE'/'FALSE', 0/1
    if isinstance(x, str):
        x = x.strip().upper()
        if x == "TRUE":
            return 1.0
        if x == "FALSE":
            return 0.0
    if x in [True, 1]:
        return 1.0
    if x in [False, 0]:
        return 0.0
    raise ValueError(f"Valeur inattendue pour 'male' : {x}")

test_df["male"] = test_df["male"].apply(convert_male_to_float).astype(np.float32)

# 3) Création robuste de la colonne image_path
def build_image_path(img_id: str) -> str | None:
    """
    img_id est de la forme '1.png', '2.png', ...
    On cherche d'abord /preprocessed_test_eliva25/1.png
    puis /preprocessed_test_eliva25/1.png.png si besoin.
    """
    img_id_str = str(img_id)

    p1 = PREPROC_TEST_DIR / img_id_str
    if p1.exists():
        return str(p1)

    p2 = PREPROC_TEST_DIR / f"{img_id_str}.png"
    if p2.exists():
        return str(p2)

    # Si aucune des deux n'existe → None
    return None

test_df["image_path"] = test_df["id"].apply(build_image_path)

missing_paths = test_df["image_path"].isna().sum()
if missing_paths > 0:
    print(f"\n {missing_paths} images prétraitées manquantes dans {PREPROC_TEST_DIR}.")
    display(test_df[test_df["image_path"].isna()].head())
else:
    print("\n Toutes les images test prétraitées ont été trouvées.")

# On ne garde que les lignes avec image_path valide
infer_df = test_df.dropna(subset=["image_path"]).copy()
print("\nNombre d'images utilisées pour l'inférence :", len(infer_df))
display(infer_df.head())


Aperçu test_eliva25.csv :


,id,male
0,1.png,False
1,2.png,False
2,3.png,True
3,4.png,False
4,5.png,False



Nombre de lignes dans test_eliva25.csv : 33

 Toutes les images test prétraitées ont été trouvées.

Nombre d'images utilisées pour l'inférence : 33


,id,male,image_path
0,1.png,0.0,/home/fabrice/Boneage/preprocessed_test_eliva2...
1,2.png,0.0,/home/fabrice/Boneage/preprocessed_test_eliva2...
2,3.png,1.0,/home/fabrice/Boneage/preprocessed_test_eliva2...
3,4.png,0.0,/home/fabrice/Boneage/preprocessed_test_eliva2...
4,5.png,0.0,/home/fabrice/Boneage/preprocessed_test_eliva2...


In [3]:
# ============================================================
# Cellule 3 — Dataset TF pour l'inférence (même preprocessing que V13)
# ============================================================

def load_and_preprocess_test(path, sex):
    """
    path : chemin vers image prétraitée (CLAHE, 300x300, 1 canal)
    sex  : 0. ou 1.
    Retour : dict inputs pour le modèle V13.
    """
    img_bytes = tf.io.read_file(path)
    img = tf.image.decode_png(img_bytes, channels=1)   # 1 canal

    # Conversion en RGB
    img = tf.image.grayscale_to_rgb(img)

    # Redimensionnement (sécurité, même si déjà 300x300)
    img = tf.image.resize(img, [IMG_SIZE, IMG_SIZE])

    img = tf.cast(img, tf.float32)
    img = preprocess_input(img)  # EfficientNetV2B3

    sex = tf.reshape(tf.cast(sex, tf.float32), (1,))

    return {"image_input": img, "sex_input": sex}

def create_test_dataset(df):
    ds = tf.data.Dataset.from_tensor_slices(
        (
            df["image_path"].values,
            df["male"].values
        )
    )

    ds = ds.map(
        lambda p, s: load_and_preprocess_test(p, s),
        num_parallel_calls=AUTOTUNE
    )

    ds = ds.batch(BATCH_SIZE).prefetch(AUTOTUNE)
    return ds

test_ds = create_test_dataset(infer_df)
print("Dataset TF pour l'inférence prêt.")


Dataset TF pour l'inférence prêt.


I0000 00:00:1764619305.643345   15183 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764619305.643493   15183 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764619305.643521   15183 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764619305.808748   15183 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1764619305.808861   15183 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2025-12-01

In [4]:
# ============================================================
# Cellule 4 — Chargement du modèle V13 pré-entraîné
# ============================================================

from tensorflow import keras

if not MODEL_V13_PATH.exists():
    raise FileNotFoundError(f"Modèle introuvable : {MODEL_V13_PATH}")

model_v13 = keras.models.load_model(
    MODEL_V13_PATH,
    compile=False  # Pas besoin de recompilation pour simple prédiction
)

print("Modèle V13 chargé depuis :", MODEL_V13_PATH)
model_v13.summary()


2025-12-01 20:02:02.316824: E tensorflow/core/util/util.cc:131] oneDNN supports DT_HALF only on platforms with AVX-512. Falling back to the default Eigen-based implementation if present.


Modèle V13 chargé depuis : /home/fabrice/Boneage/model_v13_preproc_v3.keras


Model: "boneage_regression_v13_preprocessed_v3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ image_input         │ (None, 300, 300,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ efficientnetv2-b3   │ (None, 10, 10,    │ 12,930,622 │ image_input[0][0] │
│ (Functional)        │ 1536)             │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ global_average_poo… │ (None, 1536)      │          0 │ efficientnetv2-b… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 1536)      │          0 │ global_average_p… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ sex_input           │ (None, 1)         │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ concatenate         │ (None, 1537)      │          0 │ dropout[0][0],    │
│ (Concatenate)       │                   │            │ sex_input[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense (Dense)       │ (None, 256)       │    393,728 │ concatenate[0][0] │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ batch_normalization │ (None, 256)       │      1,024 │ dense[0][0]       │
│ (BatchNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 256)       │          0 │ batch_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_1 (Dense)     │ (None, 128)       │     32,896 │ dropout_1[0][0]   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_2 (Dense)     │ (None, 1)         │        129 │ dense_1[0][0]     │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 13,358,399 (50.96 MB)

 Trainable params: 2,911,647 (11.11 MB)

 Non-trainable params: 10,446,752 (39.85 MB)

In [5]:
# ============================================================
# Cellule 5 — Prédictions d'âge osseux sur test_eliva25
# ============================================================

print("Calcul des prédictions sur le jeu test_eliva25 ...")

# Prédictions (shape : (n_samples, 1))
y_pred = model_v13.predict(test_ds, verbose=1).flatten()

print("\nTaille de y_pred :", y_pred.shape)
print("Exemple de prédictions (premiers 10) :", y_pred[:10])

# On associe les prédictions aux IDs correspondants (infer_df garde l'ordre)
infer_df["boneage"] = y_pred.astype(np.float32)

display(infer_df[["id", "male", "boneage"]].head())


Calcul des prédictions sur le jeu test_eliva25 ...


I0000 00:00:1764619348.574506   15264 service.cc:146] XLA service 0x7dba6c016f40 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1764619348.574556   15264 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2025-12-01 20:02:28.708191: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2025-12-01 20:02:29.324347: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 91200
I0000 00:00:1764619356.268320   15264 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


5/5 ━━━━━━━━━━━━━━━━━━━━ 23s 3s/step

Taille de y_pred : (33,)
Exemple de prédictions (premiers 10) : [130.8499   127.63482  150.69263  155.62437  168.80687  103.40605
 103.77087  139.5586   182.99675   82.861176]


,id,male,boneage
0,1.png,0.0,130.849899
1,2.png,0.0,127.634819
2,3.png,1.0,150.692627
3,4.png,0.0,155.624374
4,5.png,0.0,168.806870


In [6]:
# ============================================================
# Cellule 6 — Création de submission_v13.csv
# ============================================================

# DataFrame de soumission
submission_df = infer_df[["id", "boneage"]].copy()

# Optionnel : trier par ordre croissant d'ID (numérique)
def extract_num(id_str):
    # id_str est de type '1.png', '12.png', etc.
    return int(str(id_str).split(".")[0])

submission_df["id_num"] = submission_df["id"].apply(extract_num)
submission_df = submission_df.sort_values("id_num").drop(columns=["id_num"])

print("Aperçu de submission_v13.csv :")
display(submission_df.head())

# Sauvegarde
submission_df.to_csv(SUBMISSION_PATH, index=False)
print(f"\n Fichier de soumission sauvegardé sous : {SUBMISSION_PATH}")
print(f"Nombre de lignes dans la soumission : {len(submission_df)}")


Aperçu de submission_v13.csv :


,id,boneage
0,1.png,130.849899
1,2.png,127.634819
2,3.png,150.692627
3,4.png,155.624374
4,5.png,168.806870



 Fichier de soumission sauvegardé sous : /home/fabrice/Boneage/submission_v13.csv
Nombre de lignes dans la soumission : 33


In [7]:
# ============================================================
# Cellule 7 — Sanity checks 
# ============================================================

print("Résumé statistique des prédictions :")
display(submission_df["boneage"].describe())

print("\nÂge min / max prédit :",
      float(submission_df["boneage"].min()),
      "→",
      float(submission_df["boneage"].max()))


Résumé statistique des prédictions :


count     33.000000
mean     143.017151
std       23.800997
min       82.861176
25%      130.849899
50%      142.176178
75%      158.279358
max      182.996750
Name: boneage, dtype: float64


Âge min / max prédit : 82.86117553710938 → 182.9967498779297
